In [ ]:
import polars as pl
from math import pi
import random
import time
import sys
import rtsvg
rt = rtsvg.RACETrack()
#_svg_ = rt.rayIntersectsSegmentSVG((0.7,0),(0.9,0.8),(1,0),(0.8,0.8))
#rt.tile([_svg_])

In [ ]:
x = lambda: random.uniform(-1.0, 1.0)
_tiles_ = []
for i in range(16): _tiles_.append(rt.rayIntersectsSegmentSVG((x(),x()),(x(),x()),(x(),x()),(x(),x()), w=128, h=128))
rt.table(_tiles_, per_row=16, spacer=10)

In [ ]:
# Works reliably when using p0 as the end point
_tiles_ = []
for i in range(16):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _uv_       = (_p0_[0]-_xy_[0], _p0_[1]-_xy_[1])
    _tiles_.append(rt.rayIntersectsSegmentSVG(_xy_,_uv_,_p0_, _p1_, w=128, h=128))
rt.table(_tiles_, per_row=16, spacer=10)

In [ ]:
# Fails intermitently when using p1 as the end point
_tiles_ = []
for i in range(16):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _uv_       = (_p1_[0]-_xy_[0], _p1_[1]-_xy_[1])
    _tiles_.append(rt.rayIntersectsSegmentSVG(_xy_,_uv_,_p0_, _p1_, w=128, h=128))
rt.table(_tiles_, per_row=16, spacer=10)

In [ ]:
# Measure the accuracy of the rayIntersectsSegment function
_correct_, _incorrect_ = 0, 0
for i in range(1_000):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _uv_       = (_p1_[0]-_xy_[0], _p1_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_)
    if _xy_inter_ is not None: _correct_   += 1
    else:                      _incorrect_ += 1
print(f"Correct: {_correct_} Incorrect: {_incorrect_} | {_correct_/(_incorrect_+_correct_)}") # 78% correct rate for 10M iterations

In [ ]:
# Measure the accuracy of the rayIntersectsSegment function w/ slight epsilon
_correct_, _incorrect_, _ep_ = 0, 0, 10e-9 # around 10e-12, you start seeing the issue... (1 incorrect out of 1M)
# _ep_ = sys.float_info.min # smallest possible float ... you get about 78% correct rate here
for i in range(1_000):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _p_        = (_p1_[0] - _ep_ * (_p1_[0] - _p0_[0]), _p1_[1] - _ep_ * (_p1_[1] - _p0_[1]))
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_)
    if _xy_inter_ is not None: _correct_   += 1
    else:                      _incorrect_ += 1
print(f"Correct: {_correct_} Incorrect: {_incorrect_} | {_correct_/(_incorrect_+_correct_)}") # 78% correct rate for 10M iterations

In [ ]:
def rayIntersectsSegment(self, xy_ray, uv_ray, xy0_segment, xy1_segment, include_xy1_endpoint=False, epsilon=1e-9):
    x_r,  y_r  = xy_ray
    dx_r, dy_r = uv_ray
    x0,   y0   = xy0_segment
    x1,   y1   = xy1_segment    
    # Segment direction vector
    dx_s, dy_s = x1 - x0, y1 - y0
    # Compute determinant
    det = -dx_r * dy_s + dy_r * dx_s
    if abs(det) < 1e-10: return None # Lines are parallel or collinear
    # Compute parameters t and u
    t = ((x_r - x0) * dy_s - (y_r - y0) * dx_s) / det
    u = ((x_r - x0) * dy_r - (y_r - y0) * dx_r) / det
    # Check if intersection is valid (t >= 0 for ray, 0 <= u <= 1 for segment)
    if t >= 0.0:
        if include_xy1_endpoint:
            if 0.0 <= u <= 1.0+epsilon: return (x_r + t * dx_r, y_r + t * dy_r)
        else:
            if 0.0 <= u <  1.0-epsilon: return (x_r + t * dx_r, y_r + t * dy_r)
    return None

case1_correct, case1_incorrect = 0, 0
case2_correct, case2_incorrect = 0, 0
case3_correct, case3_incorrect = 0, 0
case4_correct, case4_incorrect = 0, 0
case5_correct, case5_incorrect = 0, 0
case6_correct, case6_incorrect = 0, 0

_ep_ = 1e-8 # starts falling apart if smaller now... so 1e-9 has errors...
for i in range(1_000):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    # The P1 EndPoint (endpoint flag matters here)
    _p_        = _p1_
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=False)
    if _xy_inter_ is None:     case1_correct   += 1
    else:                      case1_incorrect += 1    
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=True)
    if _xy_inter_ is not None: case2_correct   += 1
    else:                      case2_incorrect += 1

    # The P1 EndPoint with epsilon (the endpoint flag shouldn't matter here - both should return an intersection)
    _p_        = (_p1_[0] - _ep_ * (_p1_[0] - _p0_[0]), _p1_[1] - _ep_ * (_p1_[1] - _p0_[1]))
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=False)
    if _xy_inter_ is None: case3_incorrect += 1
    else:                  case3_correct   += 1
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=True)
    if _xy_inter_ is None: case4_incorrect += 1
    else:                  case4_correct   += 1

    # The P1 EndPoint with added epsilon (the endpoint flag shouldn't matter here -- neither should return an intersection)
    _p_        = (_p1_[0] + _ep_ * (_p1_[0] - _p0_[0]), _p1_[1] + _ep_ * (_p1_[1] - _p0_[1]))
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=False)
    if _xy_inter_ is None: case5_correct   += 1
    else:                  case5_incorrect += 1
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=True)
    if _xy_inter_ is None: case6_correct   += 1
    else:                  case6_incorrect += 1

print(f"Case 1: Correct: {case1_correct:8} Incorrect: {case1_incorrect:8} | {case1_correct/(case1_incorrect+case1_correct):.2f}")
print(f"Case 2: Correct: {case2_correct:8} Incorrect: {case2_incorrect:8} | {case2_correct/(case2_incorrect+case2_correct):.2f}")
print(f"Case 3: Correct: {case3_correct:8} Incorrect: {case3_incorrect:8} | {case3_correct/(case3_incorrect+case3_correct):.2f}")
print(f"Case 4: Correct: {case4_correct:8} Incorrect: {case4_incorrect:8} | {case4_correct/(case4_incorrect+case4_correct):.2f}")
print(f"Case 5: Correct: {case5_correct:8} Incorrect: {case5_incorrect:8} | {case5_correct/(case5_incorrect+case5_correct):.2f}")
print(f"Case 6: Correct: {case6_correct:8} Incorrect: {case6_incorrect:8} | {case6_correct/(case6_incorrect+case6_correct):.2f}")

In [ ]:
_x_r_,  _y_r_   = 'x' , 'y'
_dx_r_, _dy_r_  = 'u' , 'v'
_x0_, _y0_      = 'x0', 'y0'
_x1_, _y1_      = 'x1', 'y1'

_lu_            = {_x_r_:[], _y_r_:[], _dx_r_:[], _dy_r_:[], _x0_:[], _y0_:[], _x1_:[], _y1_:[], 'xint':[], 'yint':[]}
for i in range(10_000_000):
    for k in _lu_.keys(): 
        if k == 'xint' or k == 'yint': continue
        _lu_[k].append(x())

t0_loop = time.time()
for i in range(len(_lu_[_x_r_])):
    _xy_       = (_lu_[_x_r_][i], _lu_[_y_r_][i]) 
    _uv_       = (_lu_[_dx_r_][i], _lu_[_dy_r_][i])
    _p0_       = (_lu_[_x0_][i], _lu_[_y0_][i])
    _p1_       = (_lu_[_x1_][i], _lu_[_y1_][i])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_)
    if _xy_inter_ is not None:
        _lu_['xint'].append(_xy_inter_[0])
        _lu_['yint'].append(_xy_inter_[1])
    else:
        _lu_['xint'].append(None)
        _lu_['yint'].append(None)
t1_loop = time.time()

df             = pl.DataFrame(_lu_)
_uniq_         = 'A1A' # uniquifying a column name

# Single "with_columns" Version
t0_pl2 = time.time()
dx_s_op = (pl.col(_x1_) - pl.col(_x0_))
dy_s_op = (pl.col(_y1_) - pl.col(_y0_))
det_op  = (-pl.col(_dx_r_) * dy_s_op + pl.col(_dy_r_) * dx_s_op)
t_op    = (((pl.col(_x_r_) - pl.col(_x0_)) * dy_s_op        - (pl.col(_y_r_) - pl.col(_y0_)) * dx_s_op)        / det_op)
u_op    = (((pl.col(_x_r_) - pl.col(_x0_)) * pl.col(_dy_r_) - (pl.col(_y_r_) - pl.col(_y0_)) * pl.col(_dx_r_)) / det_op)
xi_op   = pl.when((t_op >= 0.0) & (u_op >= 0.0) & (u_op <= 1.0)).then(pl.col(_x_r_) + t_op * pl.col(_dx_r_)).otherwise(None)
yi_op   = pl.when((t_op >= 0.0) & (u_op >= 0.0) & (u_op <= 1.0)).then(pl.col(_y_r_) + t_op * pl.col(_dy_r_)).otherwise(None)
df      = df.with_columns(xi_op.alias('_xi2_'+_uniq_+'_'), yi_op.alias('_yi2_'+_uniq_+'_'))
t1_pl2 = time.time()

# Multi-Step Version
t0_pl = time.time()
_dx_s_, _dy_s_ = '_dx_s_'+ _uniq_ +'_', '_dy_s_' + _uniq_ + '_'
df             = df.with_columns((pl.col(_x1_) - pl.col(_x0_)).alias(_dx_s_),
                                 (pl.col(_y1_) - pl.col(_y0_)).alias(_dy_s_),)
_det_          = '_det_' + _uniq_ + '_'
_t_, _u_       = '_t_' + _uniq_ + '_', '_u_' + _uniq_ + '_'
df             = df.with_columns((-pl.col(_dx_r_) * pl.col(_dy_s_) + pl.col(_dy_r_) * pl.col(_dx_s_)).alias(_det_))
df             = df.with_columns((((pl.col(_x_r_) - pl.col(_x0_)) * pl.col(_dy_s_) - (pl.col(_y_r_) - pl.col(_y0_)) * pl.col(_dx_s_)) / pl.col(_det_)).alias(_t_),
                                 (((pl.col(_x_r_) - pl.col(_x0_)) * pl.col(_dy_r_) - (pl.col(_y_r_) - pl.col(_y0_)) * pl.col(_dx_r_)) / pl.col(_det_)).alias(_u_),)
_xi_, _yi_     = '_xi_' + _uniq_ + '_', '_yi_' + _uniq_ + '_'
df             = df.with_columns(pl.when((pl.col(_t_) >= 0.0) & (pl.col(_u_) >= 0.0) & (pl.col(_u_) <= 1.0)).then(pl.col(_x_r_) + pl.col(_t_) * pl.col(_dx_r_)).otherwise(None).alias(_xi_),
                                 pl.when((pl.col(_t_) >= 0.0) & (pl.col(_u_) >= 0.0) & (pl.col(_u_) <= 1.0)).then(pl.col(_y_r_) + pl.col(_t_) * pl.col(_dy_r_)).otherwise(None).alias(_yi_),)
t1_pl = time.time()

# polars time: 0.393/0.285 | loop time: 8.815/8.327 # first cut at the timing with 10M points... (although didn't check for correctness in the polars version...)
# ... of course, upon inspection... the implementation is not correct -- i.e., it doesn't match the xin and yint columns that were added...
# ... okay / fixed the error ... and the added a single operation version to get the xint and yint columns...
# polars time: 0.213 | polars (2) time: 0.955 | loop time:  9.342 # 10M points / M1 Pro (16G) (multistep done first)
# polars time: 0.323 | polars (2) time: 0.988 | loop time:  8.490 # 10M points / M1 Pro (16G) (multistep done first)
# polars time: 0.334 | polars (2) time: 0.763 | loop time:  9.142 # 10M points / M1 Pro (16G) (multistep done first)
# polars time: 0.126 | polars (2) time: 0.392 | loop time:  8.777 # 10M points / M1 Pro (16G) (single with_columns done first)
# polars time: 0.130 | polars (2) time: 0.754 | loop time:  8.852 # 10M points / M1 Pro (16G) (single with_columns done first)
# polars time: 0.130 | polars (2) time: 0.986 | loop time:  8.982 # 10M points / M1 Pro (16G) (single with_columns done first)
# polars time: 0.129 | polars (2) time: 0.610 | loop time:  5.365 # 10M points / 7900x (96G) (single with_columns done first)
# polars time: 1.292 | polars (2) time: 6.381 | loop time: 53.450 # 100M points / 7900x (96G) (single with_columns done first)
print(f'polars time: {t1_pl-t0_pl:.3f} | polars (2) time: {t1_pl2-t0_pl2:.3f} | loop time: {t1_loop-t0_loop:.3f}')
df

In [ ]:
# Create X,Y columns to compare every point against every other point
df = pl.DataFrame({'x':[1.0, 2.0, 5.0], 'y':[1.5, 3.0, 4.5]})
df = df.with_columns(pl.struct(['x','y']).implode().alias('_implode_')) \
       .explode('_implode_')                                            \
       .with_columns(pl.col('_implode_').struct.field('x').alias('x1'),
                     pl.col('_implode_').struct.field('y').alias('y1'))
df

In [ ]:
_lu_ = {'x':[],'y':[]}
for i in range(10_000): _lu_['x'].append(x()), _lu_['y'].append(x())
df = pl.DataFrame(_lu_)
t0 = time.time()
df = df.with_columns(pl.struct(['x','y']).implode().alias('_implode_')) \
       .explode('_implode_')                                            \
       .with_columns(pl.col('_implode_').struct.field('x').alias('x1'),
                     pl.col('_implode_').struct.field('y').alias('y1'))
t1 = time.time()
print(f'polars time: {t1-t0:.3f}')
#   1K -->   1M points / 0.007s / M1 Pro (16G)
#  10K --> 100M points / 0.334s / M1 Pro (16G)
#  20K --> 400M points / 3.635s / M1 Pro (16G) # about where 16g maxes out
#  20K --> 400M points / 3.241s / 7900x (96G)
df

In [ ]:
_dx_ = pl.col('x1') - pl.col('x')
_dy_ = pl.col('y1') - pl.col('y')
t0 = time.time()
df   = df.with_columns((     (16*(pl.arctan2(_dy_, _dx_) + pl.lit(pi))/(pl.lit(2*pi))).cast(pl.Int64)    ).alias('sector'))
t1 = time.time()
print(f'polars time: {t1-t0:.3f}')
print(df['sector'].min(), df['sector'].max())
#   1K  ->   1M points / arctan2 0.037s     / M1 Pro (16G)
#  10K  -> 100M points / arctan2 4.17s      / M1 Pro (16G)
#  10K  -> 100M points / sector calc 4.844s / M1 Pro (16G)
#  20K  -> 400M points / sector calc 9.069s / 7900x (96G)
#  10K  -> 100M points / sector calc 2.173s / 7900x (96G)
df

In [ ]:
rt.histogram(df, bin_by='sector', h=288)